<a href="https://colab.research.google.com/github/yeaeunJi/deep_learning-/blob/main/%ED%95%A9%EC%84%B1%EA%B3%B1_%EC%8B%A0%EA%B2%BD%EB%A7%9D(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 합성곱 신경망(convolutional neural network)
- 합성곱 : '두 함수 중 하나를 반전, 이동시켜가며 나머지 함수와의 곱을 연이어 적분하는 것
- CNN은 이미지 인식과 음성 인식 등 다양하게 사용되는데, 특히 이미지 인식 분야에서 딥러닝 활용기법은 거의 다 CNN을 기초로 함

## CNN 네트워크 구조
- 앞에서 본 신경망처럼 계층을 조합하여 만들 수 있는데 합성곱 계층과 풀링 계층이 새롭게 등장

- 완전연결(fully connected) : 인접하는 계층의 모든 뉴런과 결합되는 신경망
  - 책에서는 완전히 연결된 계층으로  Affine 계층을 구현했었음
  - 입력데이터 -----> Affine ---> ReLu ---> Affine ---> Softmax
- CNN의 구조 : 합성곱 계층과 풀링 계층이 추가됨
  - 입력데이터 -----> Conv(합성곱계층) ---> ReLu ---> Poling(풀링계층) ---> Conv ---> ReLU ---> Affine ---> Softmax 
  - 풀링 계층은 생략하기도 함
  - 출력에 가까운 층에서는 Affine ---> ReLU 구성 사용 가능
  - 마지막 출력 계층에서 Affine ---> Softmax 그대로 사용 

## 합성곱 계층

### 완전연결 계층의 문제점
- 인접하는 계층의 뉴런이 모두 연결되며 출력 수는 임의로 설정 가능한 계층임
- 문제점 :  데이터의 형상이 무시됨
  
  예) 입력데이터가 세로, 가로, 색상으로 구성된 3차원 데이터인 이미지일 경우
    - 완전 연결 계층에 입력 시 3차원인 데이터를 1차원 데이터로 평탄화해줘야 함
    - 3차원이 갖는 공간적 정보(예:공간적으로 가까운 픽셀값은 서로 비슷하다 등)가 사라짐

- 합성곱 계층은 이러한 데이터의 형상이 유지됨
  - 입력 데이터가 3차원이라면 다음 계층에도 3차원 데이터로 전달

- 특징 맵(feature map) : 합성곱 계층의 입출력 데이터
  - 입력 특징 맵(input feature map) : 합성곱 계층의 입력 데이터
  - 출력 특징 맵(output feature map) : 합성곱 계층의 출력 데이터


### 합성곱 연산
- 합성곱 계층에서의 합성곱 연산을 처리
- 합성곱 연산 : 이미지 처리에서 말하는 필터 연산에 해당함
- 입력데이터 --->  (합성곱) 필터 또는 커널 ---> 출력
- 합성곱 연산은 필터의 윈도우를 일정 간격으로 이동해가며 입력 데이터에 단일 곱셈 누산 계산을 함
  - 단일 곱셈 누산(fused multiply-add, FMA) : 입력과 필터에서 대응하는 원소끼리 곱한 후 그 총합을 구하여 출력의 해당 장소에 저장

- 완전연결 신경망에서의 가중치 매개변수와 편향처럼 CNN에서는 필터의 매개변수가 그동안의 가중치에 해당하고 편향 역시 존재. 편향은 필터 적용 후의 데이터에 각 원소마다 더해지며 항상 하나만 존재

### 패딩(padding)
- 합성곱 연산 수행 전 입력 데이터 주변에 특정 값으로 채우는 것
  - 1짜리 패딩을 줄 경우, (4,4) 주변을 특정값으로 둘러쌈
  - 패딩의 크기가 커질수록 출력 데이터의 크기도 커짐
- 합성곱 연산 시 출력 크기를 조정할 목적으로 사용됨
  - (4,4) 입력데이터에 (3,3) 필터를 합성곱 연산하면 그 출력값이 (2,2)가 되는데 이렇게 합성곱 연산이 진행되다보면 출력 크기가 1이 되어 더이상 합성곱 연산을 적용할 수 없게 됨
  - 패딩을 사용하여 입력데이터의 공간적 크기를 고정한 채 다음 계층에 전달 가능

### 스트라이드(stride)
- 필터를 적용하는 위치의 간격(stride는 '보폭'이란 의미라고 함)
- 스트라이드를 2로하면 필터를 적용하는 윈도우가 2칸씩 이동하며, 스트라이드의 크기가 커질수록 출력 크기는 작아짐

### 입력데이터, 패딩, 스트라이드와 출력 데이터의 관계
- 출력 데이터의 행  = (입력데이터행 + 2 * 패딩 - 필터의행크기) / 스트라이드 + 1
- 출력 데이터의 열 =  (입력데이터열 + 2 * 패딩 - 필터의열크기) / 스트라이드 + 1
- 위 수식의 결과값은 항상 정수가 나와야 함

### 3차원 데이터의 합성곱 연산
- 입력 데이터의 채널 수와 필터의 채널 수가 같아야 함
- 필터를 FN개 두면, 출력 맵도 FN개가 생성되는데 이를 모으면(FN, OHm, OW)인 블록이 나올 수 있고 이를 다음 계층으로 넘기게되면 3차원 데이터의 형상이 유지될 수 있다.
- 합성곱 연산에서도 편향이 쓰이는데 (FN, 1, 1) 형상으로 각 필터의 출력 결과에 대응 채널의 원소 모두에 더해짐

### 배치 처리
- 합성곱 연산에서 배치 처리를 하기 위해서 각 계층을 흐르는 데이터의 차원을 하나 늘려 4차+원 데이터로 저장함(데이터 수, 채널 수, 높이, 너비)

## 풀링 계층
- 풀링 : 세로, 가로 방향의 공간을 줄이는 연산으로 (x,x) 영역을 원소 하나로 집약하여 공간의 크기를 줄임
- 최대 풀링(max pooling) : 대상 영역 중 가장 큰 값을 선택하여 축약
- 평균 풀링(average pooling) : 대상 영역의 평균을 계산
- 영역 x의 값과 스트라이드의 값은 일반적으로 같은 값을 설정함

### 풀링 계층의 특징 
- 1. 학습해야 할 매개변수가 없음
  - 합성곱 계층과는 달리 대상 영역에서 최댓값 또는 평균을 취하는 등의 명확한 값을 취하기에 학습해야 할 매개변수가 존재하지 않음
- 2. 채널 수가 변하지 않음
  - 풀링 연산 시 채널 별 독립적 계산이 이루어지므로 입력 데이터의 채널 수가 그대로 출력 데이터로 내보내짐
- 3. 입력의 변화에 영향을 적게 받음
  - 입력 데이터가 조금 변한다고 풀링의 결과가 잘 변하지는 않음

## 합성곱/풀링 계층 구현하기

In [1]:
 # 4차원 배열 : CNN에서 계층 사이를 흐르는 데이터는 4차원 형태로 (데이터 수, 채널 수, 높이, 너비)를 의미
 import numpy as np

 x = np.random.rand(10, 1, 28, 28) # 무작위로 데이터 생성
 x.shape

(10, 1, 28, 28)

In [2]:
# 10개의 데이터 중 첫번째 데이터에 접근하는 방법
print(x[0].shape)
print(x[2].shape)

(1, 28, 28)
(1, 28, 28)


In [ ]:
# 첫번째 데이터의 첫 채널의 공간 데이터에 접근하는 방법
print(x[0,0])

### im2col로 데이터 전개하기
- 넘파이에서 원소에 접근할 때 for문을 사용하면 성능이 떨어진다는 단점이 존재하여 for문이 아닌 im2col(image to column)이란 편의 함수를 통해 구현 가능

- im2col 
  - 입력 데이터를 필터링(가중치 계산)하기 좋게 펼치는 함수로 3차원 입력 데이터에 이 함수를 적용하면 2차원 행렬로 변환(정확히는 4차원 데이터를 2차원으로 변환)
  - 필터 적용 영역을 앞에서부터 순서대로 1줄로 펼쳐 입력 데이터를 전개함
  - 필터 적용 영역이 겹치게 되면 im2col로 전개한 후의 원소 수가 원래 블록의 원소 수보다 증가하게 되면서 메모리를 더 많이 소모하게 될 수 있음
  - 하지만 컴퓨터는 큰 행렬 계산에 최적화되어 있으므로 선형 대수 라이브러리를 활용하여 효율을 높일 수 있음

- im2col로 입력 데이터 전개 후 합성곱 계층의 필터를 세로로 1열로 전개하고, 두 행렬의 곱을 계산 후 출력 데이터로 4차원 행렬로 reshape 진행

In [6]:
# 출처 : https://programmers.co.kr/learn/courses/57/lessons/13851
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


In [8]:
x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1, 5,5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0 )
print(col2.shape)

(9, 75)
(90, 75)


In [ ]:
# im2col 함수를 사용하여 합성곱 계층 구현
class Convolution:

  # 필터(가중치), 편향, 스트라이드, 패딩을 초기화
  def __init__(self, W, b, stride=1, pad=0) :
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad

  def forward(self, x) :
    FN, C, FH, FW = self.W.shape
    N, C, H, W = x.shape
    out_h = int(1+(H+2*self.pad- FH) / self.stride)
    out_w = int(1+(W+2*self.pad-FW) / self.stride)

    col = im2col(x, FH, FW, self.stride, self.pad)
    col_W = self.W.reshape(FN, -1).T # 필터 전개
    out = np.dot(col, col_W) + self.b

    out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2) # numpy의 transpose 함수 : 다차원 배열의 축 순서를 바꿔줌

    return out

  def backward(self, dout):
    FN, C, FH, FW = self.W.shape
    dout = dout.transpose(0,2,3,1).reshape(-1, FN)

    self.db = np.sum(dout, axis=0)
    self.dW = np.dot(self.col.T, dout)
    self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

    dcol = np.dot(dout, self.col_W.T)
    dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

    return dx

#### 합성곱 계층의 역전파 구현 시 im2col을 역으로 처리해야 하는데 아래의 함수를 사용하면 됨|

In [ ]:
# 출처 : https://github.com/WegraLee/deep-learning-from-scratch/blob/master/common/util.py
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """(im2col과 반대) 2차원 배열을 입력받아 다수의 이미지 묶음으로 변환한다.
    
    Parameters
    ----------
    col : 2차원 배열(입력 데이터)
    input_shape : 원래 이미지 데이터의 형상（예：(10, 1, 28, 28)）
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    img : 변환된 이미지들
    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]


### 풀링 계층 구현
- im2col을 사용해 입력 데이터를 전개하는 것은 합성곱 계층과 같지만 채널 쪽이 독립적이라는 다른 점이 존재

In [1]:
class Pooling :
  def __init__(self, pool_h, pool_w, stride=1, pad=0) :
    self.pool_h = pool_h
    self.pool_w = pool_w
    self.stride = stride
    self.pad = pad
  
  def forward(self, x) :
    N, C, H, W = x.shape
    out_h = int(1+(H-self.pool_h) / self.stride)
    out_w = int(1+(W-self.pool_w) / self.stride)

    # 입력 데이터 전개
    col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
    col = col.reshape(-1, self.pool_h*self.pool_w)

    # 최댓값(행렬의 최댓값)
    out = np.max(col, axis=1) # axis=1은 행방향, aixs=0은 열방향

    # 성형(적절한 모양으로 성형)
    out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

    return out
    
   def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx


## CNN 구현

In [ ]:
class SimpleConvNet :
  '''
  input_dim - 입력 데이터(채널수, 높이, 너비)의 차원
  conv_param - 합성곱 계층의 하이퍼파라미터(dict)
    - filter_num : 필터 수
    - filter_size : 필터 크기
    - stride : 스트라이드
    - pad : 패딩
  hidden_size - 은닉층(완전연결)의 뉴런 수
  output_size - 출력층(완전연결)의 뉴런 수
  weight_init_std - 초기화 때 가중치 표준편차
  '''
  def __init__(self, input_dim=(1, 28, 28), 
               conv_param={'filter_num' : 30, 'filter_size' : 5, 'pad':0, 'stride' : 1},
               hidden_size=100, output_size=10, weight_init_std=0.01) :
    filter_num = conv_param['filter_num']
    filter_size = conv_param['filter_size']
    filter_pad = conv_param['filter_pad']
    filter_stride = conv_param['filter_stride']

    input_size = input_dim[1]
    conv_output_size = (intput_size - filter_size + 2*filter_pad) / filter_stride+1 # 합성곱 계층의 출력 크기
    pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2)) # 풀링 계층의 출력 크기

    # 가중치 매개변수 초기화
    self.params  = {}
    
    # 1번째 층 : 합성곱 계층
    self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0],
                                                          filter_size, filter_size)
    self.params['b1'] = np.zeros(filter_num)

    # 2,3번쩨 층 : 완전연결 계층
    self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
    self.params['b2'] = np.zeros(hidden_size)
    self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b3'] = np.zeros(output_size)

    # CNN 구성 계층 생성
    self.layers = OrderDict() # 순서있는 dict 객체로 입력한 순서대로 저장됨
    self.layers['Conv1'] = Convolution(self.params['W1'],
                                       self.params['b1'],
                                       conv_param['stride'],
                                       conv_param['pad'])
    
    self.layers['Relu1'] = Relu()
    self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
    self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])

    self.layers['Relu2'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

    self.last_layer = SoftmaxWithLoss()
  
  # 추론을 수행하는 함수
  def predict(self, x) :
    for layer in self.layers.values() :
      x = layer.forward(x)
    return x

  def loss(self, x, t) :
    y = self.predict(x)
    return self.last_layer.forward(y, t)

  # 오차역전파법으로 기울기를 구하는 함수
  def gradient(self, x, t) :
    # 순전파
    self.loss(x,t)

    # 역전파
    dout = 1
    dout = self.last_layer.backward(dout)

    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)

    # 결과저장
    grads = {}
    grads['W1'] = self.layers['Conv1'].dW
    grads['b1'] = self.layers['Conv1'].db

    grads['W2'] = self.layers['Affine1'].dW
    grads['b2'] = self.layers['Affine1'].db

    grads['W3'] = self.layers['Affine2'].dW
    grads['b3'] = self.layers['Affine2'].db

    return grads